In [1]:
import cv2
import hand_track as ht
import time
import numpy as np
import math
import mediapipe as mp
import autopy

In [2]:
timeP=0
px, py = 0, 0
thickness = 3
eraser_thick = 15
canvas = np.zeros((480, 640, 3), np.uint8)
color = (255, 0, 0)

cam_w, cam_h = 640, 480    # camera parameters
scr_w, scr_h = autopy.screen.size()    # screen parameters


In [3]:
cap = cv2.VideoCapture(0)
tracker = ht.hand_tracker(max_num_hands=1, detection_conf=0.6, tracking_conf=0.7)

In [4]:
while True:
    ret, frame = cap.read()
    frame = tracker.detection(frame, draw=False)
    lm_coord, bbox = tracker.position(frame, draw=False)
    
    if len(lm_coord)!=0:
        idx_x, idx_y = lm_coord[8][1], lm_coord[8][2]
        mid_x, mid_y = lm_coord[12][1], lm_coord[12][2]
        
        fingers = tracker.fingers_pos()
        
        # selecting colors
        cv2.rectangle(frame, (0,100), (128,50), (255,0,0), cv2.FILLED)
        cv2.rectangle(frame, (128,100), (256,50), (0,255,0), cv2.FILLED)
        cv2.rectangle(frame, (256,100), (384,50), (0,0,255), cv2.FILLED)
        cv2.rectangle(frame, (384,100), (512,50), (255,0,255), cv2.FILLED)
        cv2.rectangle(frame, (512,100), (640,50), (0,0,0), cv2.FILLED)
        
        if fingers[1]==1 and fingers[2]==1:

            if 50 < idx_y < 100 :
                if 0<idx_x<128 :
                    color = (255,0,0)
                    thickness = 3
                elif 128<idx_x<256 :
                    color = (0,255,0)
                    thickness = 3
                elif 256<idx_x<384 :
                    color = (0,0,255)
                    thickness = 3
                elif 384<idx_x<512 :
                    color = (255,0,255)
                    thickness = 3
                elif 512<idx_x<640:
                    color = (0,0,0)
                    thickness = eraser_thick
            cv2.rectangle(frame, (idx_x, idx_y-20), (mid_x, mid_y+20), (255,255,255), 3)
            
        
        #drawing
        if fingers[1]==1 and fingers[2]==0:
            cv2.circle(frame, (idx_x, idx_y), 7, color, cv2.FILLED)
            
            if px==0 and py==0:
                px, py = idx_x, idx_y
            
            cv2.line(frame, (px, py), (idx_x, idx_y), color, thickness)
            cv2.line(canvas, (px, py), (idx_x, idx_y), color, thickness)
            px, py = idx_x, idx_y

        
    timeC = time.time()
    fps = 1/(timeC-timeP)
    timeP = timeC
    
    cv2.putText(frame, str(int(fps)), (5,50), cv2.FONT_HERSHEY_DUPLEX, 3, (0,255,0), 3)
    
    frame = cv2.addWeighted(frame, 0.7, canvas, 0.8, 0)
    cv2.imshow('frame', frame)
    cv2.imshow('canvas', canvas)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()